Please download the datasets from here: https://drive.google.com/open?id=1mNEblJS0622w5-2mB6yCyMbu7RbcfENL<br>
This should contain the following:
- __5_min_train__: training data as np array. Shape: (7500, 64, 64, 8)
- __5_min_xval__: validation data (currently unused) as np array. Shape: (1500, 64, 64, 8)
- __5_min_test__: test data (used as visual validation during training) as np array. Shape: (1000, 64, 64, 8)
- __5_min_norms__: list of floats containing the maximum pixel intensity value prior to normalization for each sequence. Shape: (10000,)
- __5_min_long_pred__: test data for sequence prediction as np array. We used it for testing after training. Shape: (1000, 64, 64, 20)
- __5_min_long_pred_norms__: list of floats containing the maximum pixel intensity value prior to normalization for each sequence for the 5_min_long_pred dataset. Shape: (1000,)
- __tgan_1/2/4-1_vx/vy_2000__: optical flow images between the last and second last frames of the input for the first 2000 sequences of the training dataset (__5_min_train__) as np array. The 1/2/4 means the length of the input sequence. We mostly used 2. Shape: (2000, 64, 64, 1)
- __germany__: Not needed. (GPS coordinates of Germany. Used for experimenting before.)

In the datasets the first axis is stands for the sample, the next two for the frame height and width and the last for the channels which is the time axis here.<br>
If data is missing or you cannot acces the drive, please write me an E-Mail: pkicsiny@gmail.com

In [1]:
import src
import keras.backend as K
import os
import numpy as np
import sys
import matplotlib.pyplot as plt
import pandas as pd

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
#model folder
sys.path.insert(0, 'C:/Users/pkicsiny/Desktop/TUM/3/ADL4CV/ADL4CV_project/models/')
#data folder
sys.path.insert(0, 'C:/Users/pkicsiny/Desktop/TUM/3/ADL4CV/data')

In [3]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0" #"" or "-1" for CPU, "0" for GPU
import tensorflow as tf
from tensorflow import keras
from keras.models import load_model
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 7567757519429999643
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 1508248780
locality {
  bus_id: 1
  links {
  }
}
incarnation: 5864431339863555789
physical_device_desc: "device: 0, name: GeForce GT 740M, pci bus id: 0000:01:00.0, compute capability: 3.5"
]


Global params.

In [5]:
past = 2
name = f"sgan_{past}-1_w"
iterations = 5000
batch_size = 64
#initialize random seed
RND = 777
np.random.seed(RND)
#set this to true if you want to use a pretrained model and load its weights from file
use_loaded = True

## Load dataset

In [ ]:
train, xval, test = src.load_datasets(past_frames=past)

Split data to inputs and ground truth images. Does data augmentation by rotations. Uses the first 2000 samples (bc. of memory issue).

In [ ]:
gan_train, gan_truth, gan_val, gan_val_truth, gan_test, gan_test_truth = src.split_datasets(
            train[:2000], xval, test, past_frames=past, augment=True)

Make labels.

In [6]:
# Adversarial ground truths
real = -np.ones((batch_size, 1))
fake = np.ones((batch_size, 1))

Noisy labels.

In [7]:
#real, fake = src.noisy_d_labels(real, fake)

Make generator but don't compile.

In [8]:
generator = src.unet((64, 64, past), dropout=0, batchnorm=True, kernel_size=4, feature_mult=1)

Make discriminator.

In [9]:
discriminator = src.spatial_discriminator(condition_shape=(64, 64, past), dropout = 0.25, batchnorm=True, wgan=True)

Inputs and outputs of the GAN.

In [10]:
frame_t = keras.layers.Input(shape=(64, 64, past))
generated = generator(frame_t)
score = discriminator([frame_t, generated])

Loss weights.

In [11]:
loss_weights = [0,1]
#L1 as metric

Make combined model: input is past sequence and output is the softmax score (Adv. loss) and the generated next frame (L1 but with 0 weight).

In [12]:
combined = keras.models.Model(inputs=[frame_t], outputs=[generated, score])

Compile discriminator with trainable weights.

In [13]:
discriminator.compile(loss=src.wasserstein_loss, optimizer=keras.optimizers.RMSprop(lr=0.00005))

Freeze discriminator weights and compile combined GAN with only the generator trainable.

In [14]:
discriminator.trainable = False
for l in discriminator.layers: l.trainable = False
combined.compile(loss=[src.custom_loss(loss="l1"), src.wasserstein_loss],
                 optimizer=keras.optimizers.RMSprop(lr=0.00005),
                 loss_weights=loss_weights)

***Using L1 loss.***


If there is a pretrained model load and use its weights. Also set discriminator weights to trainable then recompile model.

In [15]:
if use_loaded:
    combined.load_weights(sys.path[1]+name+"/"+name+"_model.h5")
    generator = combined.layers[1]
    discriminator = combined.layers[2]
    discriminator.trainable = True
    for l in discriminator.layers: l.trainable = True
    discriminator.compile(loss=src.wasserstein_loss, optimizer=keras.optimizers.RMSprop(lr=0.00005))
    discriminator.trainable = False
    for l in discriminator.layers: l.trainable = False
    combined.compile(loss=[src.custom_loss(loss="l1"), src.wasserstein_loss],
                     optimizer=keras.optimizers.RMSprop(lr=0.00005),
                     loss_weights=loss_weights)

***Using L1 loss.***


Log dict. Either append to existing log or start with an empty one.

In [16]:
if use_loaded:
    log = np.load(sys.path[1]+name+"/"+name+"_log.npy").item()
else:
    log = {"g_loss":[],
           "d_loss":[],
           "g_metric":[],
           "d_metric":[],
           "d_loss_real":[],
           "d_loss_fake":[],
           "d_test_real":[],
           "d_test_fake":[]}

## Training

In [ ]:
for it in range(iterations):
    
    if (it % 1000) < 25 or it % 500 == 0: # 25 times in 1000, every 500th
        d_iters = 100
    else:
        d_iters = 10

    discriminator.trainable = True
    for l in discriminator.layers: l.trainable = True
    for d_it in range(d_iters): 
        idx = np.random.choice(gan_truth.shape[0], batch_size, replace=False)
        real_imgs = gan_truth[idx]
        training_batch = gan_train[idx]
        generated_imgs = generator.predict(training_batch) 

        for l in discriminator.layers:
            weights = l.get_weights()
            weights = [np.clip(w, -0.01, 0.01) for w in weights]
            l.set_weights(weights)
        
        #train on real
        d_loss_real = discriminator.train_on_batch([training_batch, real_imgs], real)
        #train on fake
        d_loss_fake = discriminator.train_on_batch([training_batch, generated_imgs], fake)
        
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)  
    discriminator.trainable = False 
    for l in discriminator.layers: l.trainable = False
    
    idx = np.random.choice(gan_truth.shape[0], batch_size, replace=False)
    training_batch = gan_train[idx]
    training_truth = gan_truth[idx]
    
    g_loss = combined.train_on_batch(training_batch, [training_truth, real])
    
    log["g_loss"].append(g_loss)
    log["d_loss"].append(d_loss)
    log["g_metric"].append(g_loss[1])
    log["d_loss_real"].append(d_loss_real)
    log["d_loss_fake"].append(d_loss_fake)
    
    print(f"\033[1m {it} [D loss: {d_loss}]\033[0m \n"+
          f"\033[1m {it} [G loss: {g_loss[0]}, G obj.: {g_loss[1]}, G wgan.: {g_loss[2]}]\033[0m \n"+
          f"\033[1m {it} [real loss: {d_loss_real}, fake loss: {d_loss_fake}]\033[0m")
    if (it%100 == 0 or it == iterations-1) and it > 0:
        src.sample_images(it, gan_test, gan_test_truth, past, generator)
        src.update_output("")
        src.plot_training_curves(log, it, name, True)

## Save features

Save model history

In [ ]:
np.save(name+"_log",log)

Save model weights

In [ ]:
combined.save_weights(name+"_model.h5")

Load model and predict

Predict future frames. Loads a 20 long sequence with 1000 sequence samples.

In [ ]:
sequence_test = src.load_datasets(prediction=True)

In [ ]:
sequence_test = src.augment_data(sequence_test[:100])

In [ ]:
sequence_test.shape

In [ ]:
#test
predictions = {}
past_frames = sequence_test[...,0:past]
test_truth = sequence_test[...,past:past+1]

In [ ]:
gen = combined.layers[1]

In [ ]:
for t in range(5):
    print(t)
    future = gen.predict(past_frames, batch_size=64)
    predictions[f"{t}"] = future
    past_frames = np.concatenate((past_frames[:,:,:,1:], predictions[f"{t}"]), axis=-1)
    test_truth = sequence_test[...,past+1+t:past+2+t]

Save example predictions

In [ ]:
src.save_examples(name, sequence_test, predictions, past, samples=[33,54]) # 33, 46, 54

Renormalize intensity values


In [ ]:
norms = np.load(sys.path[0]+"/5min_norms_compressed.npz")["arr_1"]

# *4 bc of augmentaion (it concats the frames so the 0th 1000th 2000th and 3000th are the same sample just rotated)
test_norms = list(norms[9000:])*4

#renormalize test samples
renormalized_test = np.array([sample * np.array(test_norms)[i] for i, sample in enumerate(sequence_test)])
renormalized_predictions = np.transpose((np.array([[sample * np.array(test_norms)[i] for i, sample in enumerate(predictions[key])] for key in ['0', '1', '2', '3', "4"]])[:,:,:,:,0]), (1,2,3,0))


Calculate pixel intensities back to dBZ and from there to mm/h. <br>
Sources: <br>
- https://www.dwd.de/DE/leistungen/radolan/radolan_info/radolan_radvor_op_komposit_format_pdf.pdf?__blob=publicationFile&v=11 (page 10)
- <https://web.archive.org/web/20160113151652/http://www.desktopdoppler.com/help/nws-nexrad.htm#rainfall%20rates>


In [ ]:
#dBZ
dBZ_t = renormalized_test*0.5 - 32.5
dBZ_p = renormalized_predictions*0.5 - 32.5
#mm/h
I_t = (0.005*10**(0.1*dBZ_t))**(0.625)
I_p = (0.005*10**(0.1*dBZ_p))**(0.625)

thresholds = [0.5]
scores = {}
for t in range(5): # loop over the predictions (4)
    print(t)
    for s in thresholds: # make a dict entry for each threshold score
        scores[f"pred_{t+1}"] = src.calculate_skill_scores(I_p[...,t:t+1],
                                                          I_t[...,past+t:past+1+t],
                                                          x=I_t[...,past-1:past],
                                                          threshold=s)

In [ ]:
np.save(name+"_scores",scores)

In [ ]:
scores.keys()

In [ ]:

np.mean(pd.Series(scores["pred_1"]["corr_to_input"]).dropna())

In [ ]:
np.mean(pd.Series(scores["pred_4"]["far"]).dropna())